PAIR ESTADISTICA INFERENCIAL MODULO 3

In [131]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [132]:
df = pd.read_csv("world_data_full_apply_nulos.csv", index_col = 0)
df.head()

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.230,323.0,32.49,93.0,Kabul,8.672,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28.748,9.0,11.78,355.0,Tirana,4.536,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,NaN,55.7,18.6,36.6,12.33,NaN,41.153332,20.168331,Europe
2,Algeria,18.0,DZ,17.4,NaN,317.0,24.28,213.0,Algiers,150.006,151.36,2.0,DZD,3.02,0.8,0.28,1.699882e+11,109.9,51.4,20.1,Algiers,76.7,112.0,0.95,Arabic,28.1,1.72,NaN,41.2,37.2,66.1,11.70,NaN,28.033886,1.659626,Africa
3,Andorra,164.0,AD,40.0,468.000,NaN,7.20,376.0,Andorra la Vella,469.000,NaN,NaN,EUR,1.27,34.0,1.51,3.154058e+09,106.4,NaN,2.7,Andorra la Vella,NaN,NaN,6.63,Catalan,36.4,3.33,77.142,NaN,NaN,NaN,NaN,67.873,42.506285,1.521801,Europe
4,Angola,26.0,AO,47.5,NaN,117.0,40.73,244.0,Luanda,34.693,261.73,17.1,AOA,5.52,46.3,0.97,9.463542e+10,113.5,9.3,51.6,Luanda,60.8,241.0,0.71,Portuguese,33.4,0.21,NaN,77.5,9.2,49.1,6.89,NaN,-11.202692,17.873887,Africa


# Ejercicios intervalo de confianza

In [133]:

def intervalo_confianza(dataframe, columna, nivel_confianza = 0.95): # si no se especifica dejamos un nivel confianza 95% por defecto
    
    """
    Calcula el intervalo de confianza para una columna específica en un DataFrame.

    Parámetros:
    -----------
    dataframe : DataFrame
        El DataFrame que contiene los datos de la muestra.
    columna : str
        El nombre de la columna para la cual deseas calcular el intervalo de confianza.
    nivel_confianza : float, opcional
        El nivel de confianza deseado para el intervalo de confianza (valor predeterminado es 0.95).

    Salida:
    -------
    None
        La función imprime en la consola la siguiente información:
        - La media muestral de la columna especificada.
        - La desviacion tipica muestral de la columna especificada
        - El nivel de confianza utilizado.
        - El intervalo de confianza calculado, que incluye el límite inferior y el límite superior.
    """

    # Calcular la media y el error estándar de la muestra
    media = dataframe[columna].mean()
    desviacion_estandar = dataframe[columna].std()

    # calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
    tamaño_muestra = len(df[columna]) 
    grados_libertad = tamaño_muestra - 1
    
    
    # Calcular el intervalo de confianza al 95%
    confidence_interval = stats.t.interval(
        confidence=nivel_confianza,         # Nivel de confianza
        df = grados_libertad,             # Grados de libertad. Es para corregir el sesgo. Siempre lo ponemos así
        loc= media,  # Media de la muestra
        scale = desviacion_estandar/ np.sqrt(tamaño_muestra))  # Adaptamos desviación estándar a la muestra


    print(f"Intervalo de Confianza para {columna}:")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Intervalo de Confianza: {confidence_interval}")
    

In [134]:
def normalidad (alfa, columna, dataframe):
    
    normalidad = True 
    # predefinimos una variable normalidad que asignaremos como true pero verificaremos con las pruebas de shapiro o kolmogorov
    
    if len(dataframe[columna]) > 50:
        p_valor_norm = stats.kstest(dataframe[columna], 'norm').pvalue  # Kolmogorov-Smirnov si n > 50
    else:
        p_valor_norm = stats.shapiro(dataframe[columna]).pvalue  # Shapiro-Wilk si n <= 50
            

    if p_valor_norm > alfa:
            print(f"La distribución de la columna {columna} es normal.")
            return normalidad 
    else:
        print(f"La distribución de la columna {columna} no es normal.")
        normalidad = False
        return normalidad

In [135]:
    # Prueba de igualdad de varianzas
def igualdad_varianzas (alfa, normalidad, *args):
    
   varianzas_iguales = True
    
   if normalidad:
        p_valor_varianza = stats.bartlett(*args).pvalue  # Test de Bartlett si los datos son normales
   else:
        p_valor_varianza = stats.levene(*args, center="median").pvalue  # Test de Levene si no son normales

   if p_valor_varianza > alfa:
        print(f"Las varianzas entre los grupos son homogeneas")
        return varianzas_iguales
   else:
        print(f"Las varianzas entre grupos no son iguales")
        varianzas_iguales = False
        return varianzas_iguales

In [136]:

def prueba_hipotesis(alfa, *args):
    """
    Realiza una prueba de hipótesis para comparar grupos.
    1. Primero verifica si los datos son normales usando el test de Shapiro-Wilk o Kolmogorov-Smirnov.
    2. Si los datos son normales, usa Bartlett para probar igualdad de varianzas. Si no son normales, usa Levene.
    3. Si las varianzas son iguales, usa el t-test de Student; si no, usa la versión de Welch.
    4. Si los datos no son normales, usa el test de Mann-Whitney (alternativa no paramétrica).

    Parámetros:
    *args: listas o arrays con los datos de cada grupo.

    Retorna:
    dict con resultados del test de normalidad, varianza e hipótesis.
    """
    
    # Verificar si hay al menos dos grupos
    if len(args) < 2:
        raise ValueError("Se necesitan al menos dos conjuntos de datos para realizar la prueba.")
    
    # Comprobar normalidad en cada grupo
    normalidad = []
    for grupo in args:
        if len(grupo) > 50:
            p_valor_norm = stats.kstest(grupo, 'norm').pvalue  # Kolmogorov-Smirnov si n > 50
        else:
            p_valor_norm = stats.shapiro(grupo).pvalue  # Shapiro-Wilk si n <= 50
        normalidad.append(p_valor_norm > alfa)

    datos_normales = all(normalidad)  # True si todos los grupos son normales

    # Prueba de igualdad de varianzas
    if datos_normales:
        p_valor_varianza = stats.bartlett(*args).pvalue  # Test de Bartlett si los datos son normales
    else:
        p_valor_varianza = stats.levene(*args, center="median").pvalue  # Test de Levene si no son normales

    varianzas_iguales = p_valor_varianza > alfa

    # Aplicar el test adecuado
    if datos_normales:
        if varianzas_iguales:
            p_valor = stats.ttest_ind(*args, equal_var=True).pvalue
            test_usado = "t-test de Student (varianzas iguales)"
        else:
            p_valor = stats.ttest_ind(*args, equal_var=False).pvalue
            test_usado = "t-test de Welch (varianzas desiguales)"
    else:
        try: # si tengo dos grupos
            p_valor = stats.mannwhitneyu(*args).pvalue
            test_usado = "Mann-Whitney U (prueba no paramétrica)"
        except: # si tengo mas de dos grupos
            p_valor = stats.kruskal(*args).pvalue
            test_usado = "Kruskal Wallis"



    # Nivel de significancia
    alfa = alfa

    # Resultados
    resultado = {
        "Test de Normalidad": normalidad,
        "Datos Normales": datos_normales,
        "p-valor Varianza": p_valor_varianza,
        "Varianzas Iguales": varianzas_iguales,
        "Test Usado": test_usado,
        "p-valor": p_valor,
        "Conclusión": "Rechazamos H0 (Diferencias significativas)" if p_valor < alfa else "No se rechaza H0 (No hay diferencias significativas)"
    }

    # Imprimir resultados de manera más clara
    print("\n📊 **Resultados de la Prueba de Hipótesis** 📊")
    print(f"✅ Test de Normalidad: {'Sí' if datos_normales else 'No'}")
    print(f"   - Normalidad por grupo: {normalidad}")
    print(f"✅ Test de Varianza: {'Iguales' if varianzas_iguales else 'Desiguales'} (p = {p_valor_varianza:.4f})")
    print(f"✅ Test aplicado: {test_usado}")
    print(f" p-valor: {p_valor:.4f}")
    print(f"🔍 Conclusión: {resultado['Conclusión']}\n")

    return resultado

In [137]:
# 1.1. Intervalo de Confianza para la Esperanza de Vida
diccionario_continentes = {}
for continente in df["continent"].unique():
    diccionario_continentes[continente] = df[df["continent"] == continente]
    
for k, v in diccionario_continentes.items():
    
    if type(k) == float: 
        pass
    else: 
        print(f"Analisis para el continente {k}")
        print(intervalo_confianza(v, "life_expectancy", 0.95 ))
        print("-------------")

Analisis para el continente Asia
Intervalo de Confianza para life_expectancy:
Media Muestral: 73.78
Nivel de Confianza: 0.95
Intervalo de Confianza: (73.10465234467596, 74.45898401896041)
None
-------------
Analisis para el continente Europe
Intervalo de Confianza para life_expectancy:
Media Muestral: 79.2
Nivel de Confianza: 0.95
Intervalo de Confianza: (78.69086882371957, 79.71378233907114)
None
-------------
Analisis para el continente Africa
Intervalo de Confianza para life_expectancy:
Media Muestral: 63.72
Nivel de Confianza: 0.95
Intervalo de Confianza: (62.87745851065992, 64.57254148934007)
None
-------------
Analisis para el continente Central America
Intervalo de Confianza para life_expectancy:
Media Muestral: 74.69
Nivel de Confianza: 0.95
Intervalo de Confianza: (74.19229653292875, 75.18770346707122)
None
-------------
Analisis para el continente South America
Intervalo de Confianza para life_expectancy:
Media Muestral: 74.82
Nivel de Confianza: 0.95
Intervalo de Confianza: 

Puede ser útil para: 

- Los resultados pueden ayudar a las organizaciones de salud a identificar áreas donde se necesita mejorar la atención médica y las políticas de salud.
- Priorización de Recursos: Permite asignar recursos a regiones con esperanzas de vida más bajas y enfocar los esfuerzos en mejorar la calidad de vida.

In [138]:
# 1.2.Intervalo de Confianza para el PIB
for k, v in diccionario_continentes.items():
    
    if type(k) == float: 
        pass
    else: 
        print(f"Analisis para el continente {k}")
        print(intervalo_confianza(v, "gdp", 0.90))
        print("-------------")

Analisis para el continente Asia
Intervalo de Confianza para gdp:
Media Muestral: 837195265650.86
Nivel de Confianza: 0.9
Intervalo de Confianza: (473570843505.0194, 1200819687796.7078)
None
-------------
Analisis para el continente Europe
Intervalo de Confianza para gdp:
Media Muestral: 486748190304.24
Nivel de Confianza: 0.9
Intervalo de Confianza: (387063388626.3799, 586432991982.0984)
None
-------------
Analisis para el continente Africa
Intervalo de Confianza para gdp:
Media Muestral: 46080559143.87
Nivel de Confianza: 0.9
Intervalo de Confianza: (35695569821.91907, 56465548465.81678)
None
-------------
Analisis para el continente Central America
Intervalo de Confianza para gdp:
Media Muestral: 88479927703.1
Nivel de Confianza: 0.9
Intervalo de Confianza: (55661658981.40638, 121298196424.79361)
None
-------------
Analisis para el continente South America
Intervalo de Confianza para gdp:
Media Muestral: 298433685548.77
Nivel de Confianza: 0.9
Intervalo de Confianza: (240088250591.0

- Política Económica: Ayuda a los economistas y gobiernos a comprender la variabilidad en la economía europea y tomar decisiones informadas sobre políticas fiscales y comerciales.

- Inversiones: Proporciona información valiosa para inversores y empresas que deseen expandirse.

In [139]:
# Prueba de Hipótesis
# 2.1: Test de Normalidad :Verifica si la variable "life_expectancy" (esperanza de vida) sigue una distribución normal 
# usando el test de Shapiro-Wilk o Kolmogorov-Smirnov según el tamaño de la muestra.
normalidad (0.05, 'life_expectancy', df)


La distribución de la columna life_expectancy no es normal.


False

In [140]:
# 2.2: Test de Igualdad de Varianzas: Comprueba si la varianza del PIB (GDP) es igual en países de Europa y África usando el test 
# de Bartlett (si los datos son normales) o Levene (si no son normales).

europa = df[df['continent'] == 'Europe']
africa = df[df['continent'] == 'Africa']

normalidad_europa = normalidad(0.05, 'gdp', europa)
normalidad_africa = (0.05, 'gdp', africa)

if normalidad_africa and normalidad_europa:
    normalidad = True
else:
    normalidad = False

igualdad_varianzas (0.05, normalidad, europa['gdp'], africa['gdp'])
    



La distribución de la columna gdp no es normal.
Las varianzas entre grupos no son iguales


False

In [141]:
# 2.3: Comparación de Medias (t-Test de Student o Mann-Whitney U): 
# ¿La esperanza de vida difiere significativamente entre países de Europa y Asia?

europa = df[df['continent'] == 'Europe']['life_expectancy']
asia = df[df['continent'] == 'Asia']['life_expectancy']

prueba_hipotesis(0.05, europa, asia)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Desiguales (p = nan)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
 p-valor: nan
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': nan,
 'Varianzas Iguales': False,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'p-valor': nan,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}

In [142]:
df['continent'].unique()

array(['Asia', 'Europe', 'Africa', 'Central America', 'South America',
       'Oceania', 'North America', nan], dtype=object)

In [143]:
# 2.4: Prueba de Hipótesis sobre Proporciones: ¿Hay una diferencia significativa en la tasa de desempleo entre los países de 
# Europa y América?

europa = df[df['continent'] == 'Europe']['unemployment_rate']
america = df[df['continent'].isin(['Central America', 'South America', 'North America'])]['unemployment_rate']

prueba_hipotesis(0.05, europa, america)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Desiguales (p = nan)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
 p-valor: nan
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': nan,
 'Varianzas Iguales': False,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'p-valor': nan,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}

In [150]:
df['country']

0      Afghanistan
1          Albania
2          Algeria
3          Andorra
4           Angola
          ...     
190      Venezuela
191        Vietnam
192          Yemen
193         Zambia
194       Zimbabwe
Name: country, Length: 195, dtype: object

In [151]:
# 2.5: Comparación de Medias en Precios de Gasolina : ¿El precio de la gasolina (gasoline_price) es significativamente 
# diferente entre países con un PIB alto y PIB bajo?

def categorizar_pib(pib):
    if pib < 4 * 10 ** 10:
        return 'PIB bajo'
    else:
        return 'PIB alto'
    
df['pib_categorizado']= df['gdp'].apply(categorizar_pib)

    

In [155]:
pib_alto = df[df['pib_categorizado'] == 'PIB alto']['gasoline_price']
pib_bajo = df[df['pib_categorizado'] == 'PIB bajo']['gasoline_price']

In [156]:
prueba_hipotesis(0.05, pib_alto, pib_bajo)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Desiguales (p = nan)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
 p-valor: nan
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': nan,
 'Varianzas Iguales': False,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'p-valor': nan,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}

In [146]:
# BONUS.1. Intervalo de Confianza para la Tasa de Natalidad
df_asia = df[df["continent"] == "Asia"]

intervalo_confianza(df_asia, "birth_rate", 0.99)

Intervalo de Confianza para birth_rate:
Media Muestral: 18.81
Nivel de Confianza: 0.99
Intervalo de Confianza: (17.586537299323094, 20.035735427949636)



- Planificación Demográfica: Las agencias gubernamentales pueden utilizar estos resultados para planificar servicios de salud materna y programas de educación sexual.

- Predicción de Población: Ayuda a estimar la futura población de la región, lo que es importante para la planificación de infraestructura y servicios públicos.

In [147]:
# BONUS.2. Intervalo de Confianza para la Tasa de Desempleo
df_oceania = df[df["continent"] == "Oceania"]
intervalo_confianza(df_oceania, "unemployment_rate", 0.95)

Intervalo de Confianza para unemployment_rate:
Media Muestral: 3.79
Nivel de Confianza: 0.95
Intervalo de Confianza: (3.444464814578132, 4.143035185421867)


- Políticas de Empleo: Los gobiernos pueden utilizar esta información para desarrollar políticas de empleo efectivas y programas de capacitación.
- Tendencias Económicas: Permite a los analistas identificar tendencias económicas y evaluar la salud económica de la región.

In [149]:
# Intervalo de Confianza para la Tasa de Impuestos
df_africa = df[df["continent"] == "Africa"]
intervalo_confianza(df_africa, "total_tax_rate", 0.90)

Intervalo de Confianza para total_tax_rate:
Media Muestral: 47.77
Nivel de Confianza: 0.9
Intervalo de Confianza: (44.35416966138565, 51.187006809202565)


- Recaudación de Fondos: Ayuda a los ministerios de finanzas a planificar la recaudación de impuestos y evaluar si las tasas son efectivas y justas.

- Equidad Fiscal: Permite evaluar si diferentes grupos de contribuyentes están siendo gravados de manera justa y equitativa.